# Полный baseline для Lamoda Bootcamp

В этом ноутбуке реализован полный baseline для анализа отзывов и генерации тегов. Решение включает очистку текста, кластеризацию отзывов, извлечение ключевых аспектов и классификацию с использованием классических методов и более продвинутых моделей.

> **Важно:** для работы русскоязычных трансформеров требуется интернет‑доступ для загрузки моделей. В данном окружении интернет ограничен, поэтому продвинутый модуль приведён в виде шаблона.

In [ ]:
# При необходимости установите дополнительные библиотеки.
# !pip install -U scikit-learn nltk hdbscan sentence-transformers transformers pymorphy2
pass

In [ ]:
import os
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Загрузка данных

Предполагается, что датасет находится в файле `lamoda_short.csv` и содержит колонку с текстом отзывов. Если файл не найден, создаётся небольшая демонстрационная выборка.

In [ ]:

DATA_PATH = '/home/oai/share/lamoda_short.csv'
if os.path.exists(DATA_PATH):
    df = pd.read_csv(DATA_PATH)
    possible_text_cols = ['review_text', 'text', 'comment', 'review']
    text_col = next((c for c in possible_text_cols if c in df.columns), df.columns[0])
    print(f'Загружено {len(df)} строк. Используемая колонка: {text_col}')
else:
    # Демонстрационные данные
    data = {
        'text': [
            'Это красное платье из хлопка, очень понравилось!',
            'Качество обуви ужасное, порвалась через неделю.',
            'Отличный свитер, мягкий и тёплый.',
            'Размер не совпадает, слишком мало.',
            'Сумка выглядит дорого и стильно, рекомендую.',
            'Доставка задержалась, сервис разочаровал.'
        ]
    }
    df = pd.DataFrame(data)
    text_col = 'text'
    print('Используется игрушечный набор данных.')

# Приводим к единой колонке
df = df[[text_col]].rename(columns={text_col: 'text'})
df.dropna(inplace=True)
df.head()


## Предобработка текста

Очистка включает приведение к нижнему регистру, удаление цифр, пунктуации и лишних пробелов. Можно расширить функцию лемматизацией, установив `pymorphy2`.

In [ ]:

def clean_text(text: str) -> str:
    text = str(text).lower()
    # удаляем все символы кроме букв и пробелов
    text = re.sub(r'[^а-яa-z\s]', ' ', text)
    # удаляем лишние пробелы
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# применяем очистку


df['clean_text'] = df['text'].apply(clean_text)
df.head()


## Классическая классификация

Если в датасете есть колонка `label` с разметкой, можно обучить модель классификации. Используются словесные и символьные TF‑IDF и линейный SVM.

In [ ]:

if 'label' in df.columns:
    X = df['clean_text']
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    word_vect = TfidfVectorizer(ngram_range=(1,2), max_features=50000, token_pattern=r'\w+')
    char_vect = TfidfVectorizer(analyzer='char', ngram_range=(3,5), max_features=50000)
    vectorizer = FeatureUnion([('word', word_vect), ('char', char_vect)])
    
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    model = LinearSVC()
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
else:
    print('В датасете нет колонки label — классификация пропущена.')


## Кластеризация отзывов и извлечение аспектов

Для группировки отзывов используются TF‑IDF признаки и алгоритм KMeans. Для каждого кластера вычисляются наиболее частотные n‑граммы, которые можно использовать как шаблонные теги.

In [ ]:

# Векторизация для кластеризации
vect = TfidfVectorizer(ngram_range=(1,2), max_features=30000, token_pattern=r'\w+')
X_vec = vect.fit_transform(df['clean_text'])

# Определяем число кластеров: если отзывов мало, используем меньшее число
n_clusters = 5 if len(df) > 50 else max(2, len(df)//2)
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(X_vec)

# Функция для извлечения топ-тегов для кластера
def get_top_terms(cluster_label, top_n=10):
    mask = df['cluster'] == cluster_label
    texts = df.loc[mask, 'clean_text']
    local_vect = TfidfVectorizer(ngram_range=(1,2), max_features=5000, token_pattern=r'\w+')
    X_local = local_vect.fit_transform(texts)
    sums = X_local.sum(axis=0)
    freqs = [(term, sums[0, idx]) for term, idx in local_vect.vocabulary_.items()]
    top_terms = sorted(freqs, key=lambda x: x[1], reverse=True)[:top_n]
    return [term for term, _ in top_terms]

# Выводим теги для каждого кластера
cluster_tags = {}
for cl in sorted(df['cluster'].unique()):
    tags = get_top_terms(cl, top_n=8)
    cluster_tags[cl] = tags
    print(f'Кластер {cl}: {", ".join(tags)}')


## Шаблон продвинутой модели (BERT)

Ниже приведён пример тонкой настройки русскоязычной модели BERT для классификации отзывов. Запустите эту ячейку в окружении с доступом к интернету и установленными `transformers` и `datasets`.

In [ ]:

# from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
# from datasets import Dataset
# 
# model_name = 'DeepPavlov/rubert-base-cased'
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# dataset = Dataset.from_pandas(df[['clean_text','label']])
# 
# def preprocess_examples(batch):
#     return tokenizer(batch['clean_text'], truncation=True, padding='max_length', max_length=128)
# 
# tokenized_dataset = dataset.map(preprocess_examples, batched=True)
# tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2, stratify_by_column='label')
# 
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(set(df['label'])))
# 
# training_args = TrainingArguments(
#     output_dir='./results',
#     evaluation_strategy='epoch',
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01,
# )
# 
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset['train'],
#     eval_dataset=tokenized_dataset['test'],
# )
# trainer.train()
# trainer.evaluate()

pass
